In [16]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

# 1. Prepare list of mountains on the earth

In [2]:
# get the response in the form of html
wikiurl="https://en.wikipedia.org/wiki/List_of_mountains_by_elevation"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)

200


In [3]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
indiatable=soup.find_all('table',{'class':"wikitable"})

In [4]:
df = pd.DataFrame()
for html_table in indiatable:
    df = df.append(pd.read_html(str(html_table))[0])
print(df.shape)
df.head(5)

(1603, 5)


,Mountain,Metres,Feet,Range,Location and Notes
0,Mount Everest,8848.0,29029,Himalayas,Nepal/China
1,K2,8611.0,28251,Karakoram,Pakistan/China
2,Kangchenjunga,8586.0,28169,Himalayas,Nepal/India
3,Lhotse,8516.0,27940,Himalayas,Nepal – Climbers ascend Lhotse Face in climbin...
4,Makalu,8485.0,27838,Himalayas,Nepal


In [5]:
df[["Mountain"]].drop_duplicates().to_csv("raw_data/list_of_earth_mountains.csv", index=False)

# 2. Parse hikers forum

In [51]:
# get the response in the form of html
wikiurl="https://community.ricksteves.com/travel-forum/beyond-europe/most-beautiful-mountain"
response=requests.get(wikiurl)
print(response.status_code)

200


In [57]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
indiatable=soup.find_all('p')

In [58]:
list_of_sentences_blog = []
for row in indiatable:
    row = str(row)
    row = re.sub(r"[\<].*?[\>]", '', str(row))
    if ("http" not in row) & (len(row) > 10):
        list_of_sentences_blog.extend([sentence.strip() for sentence in row.split("\n")])

In [59]:
list_of_sentences_blog[:5]

['I need a lift today, and thought I would start a conversation about beautiful mountains.  The thought was triggered by a photo of the Matterhorn that came in an email from a travel company.  I have heard the Matterhorn described as “the most beautiful mountain in the world” and having been privileged to see her up close for several consecutive days while hiking around Zermatt, I might agree.  The shape is graceful and unforgettable.',
 'But in my mind the Matterhorn has strong competition from our local volcano Mt. Rainier, especially in summer, with her green and flowery Paradise meadows capped by a mostly-snow covered summit.  Also from The Great One, Denali, which is huge and impressive when seen from the Stony Overlook or from the Kantishna area.  When we are in Denali National Park at our favorite wilderness lodge, and this mountain comes out from behind the clouds, all I want to do is sit and gaze at him.  In June this can occur around midnight, when the mountain is tinged with

In [60]:
with open("raw_data/hiking_forums.txt", "a") as f:
    for row in list_of_sentences_blog:
        f.write(row + ".\n")